# 数据扩充_3 主动学习、半监督学习
### 2020年2月8日 WenQLi

## 一、概述

监督学习指的是训练样本包含标记信息的学习任务，例如：常见的分类与回归算法；
无监督学习则是训练样本不包含标记信息的学习任务，例如：聚类算法。
在实际生活中，常常会出现一部分样本有标记和较多样本无标记的情形，例如：做网页推荐时需要让用户标记出感兴趣的网页，但是少有用户愿意花时间来提供标记。若直接丢弃掉无标记样本集，使用传统的监督学习方法，常常会由于训练样本的不充足，使得其刻画总体分布的能力减弱，从而影响了学习器泛化性能。那如何利用未标记的样本数据呢？

主动学习：若我们先使用有标记的样本数据集训练出一个学习器，再基于该学习器对未标记的样本进行预测，从中挑选出不确定性高或分类置信度低的样本来咨询专家并进行打标，最后使用扩充后的训练集重新训练学习器，这样便能大幅度降低标记成本，这便是主动学习（active learning），其目标是使用尽量少的/有价值的咨询来获得更好的性能。

## 二、基本假设

要利用未标记样本，必然要做一些将未标记样本所揭示的数据分布信息与类别标记相联系的假设。
### 1）平滑假设(Smoothness Assumption)：
位于稠密数据区域的两个距离很近的样例的类标签相似，也就是说，当两个样例被稠密数据区域中的边连接时，它们在很大的概率下有相同的类标签；相反地，当两个样例被稀疏数据区域分开时，它们的类标签趋于不同。
### 2）聚类假设(Cluster Assumption)：
当两个样例位于同一聚类簇时，它们在很大的概率下有相同的类标签。这个假设的等价定义为低密度分离假设(Low Sensity Separation Assumption)，即分类决策边界应该穿过稀疏数据区域，而避免将稠密数据区域的样例分到决策边界两侧。
聚类假设是指样本数据间的距离相互比较近时，则他们拥有相同的类别。根据该假设，分类边界就必须尽可能地通过数据较为稀疏的地方，以能够避免把密集的样本数据点分到分类边界的两侧。
例如，Joachims提出的转导支持向量机算法（TSVM），在训练过程中，算法不断修改分类超平面并交换超平面两侧某些未标记的样本数据的标记，使得分类边界在所有训练数据上最大化间隔，从而能够获得一个通过数据相对稀疏的区域，又尽可能正确划分所有有标记的样本数据的分类超平面。
### 3）流形假设(Manifold Assumption)：
将高维数据嵌入到低维流形中，当两个样例位于低维流形中的一个小局部邻域内时，它们具有相似的类标签。
流形假设，假设数据分布在一个流形结构上，临近的样本拥有相似的输出值，临近通常用相似度来刻画。流形假设可以看作聚类假设的推广。
流形假设的主要思想：同一个局部邻域内的样本数据具有相似的性质，因此其标记也应该是相似。这一假设体现了决策函数的局部平滑性。
#### 和聚类假设的主要不同是:
#### 1）聚类假设主要关注的是整体特性，流形假设主要考虑的是模型的局部特性。
在该假设下，未标记的样本数据就能够让数据空间变得更加密集，从而有利于更加标准地分析局部区域的特征，也使得决策函数能够比较完满地进行数据拟合。流形假设有时候也可以直接应用于半监督学习算法中。
例如，Zhu 等人利用高斯随机场和谐波函数进行半监督学习，首先利用训练样本数据建立一个图，图中每个结点就是代表一个样本，然后根据流形假设定义的决策函数的求得最优值，获得未标记样本数据的最优标记；Zhou 等人利用样本数据间的相似性建立图，然后让样本数据的标记信息不断通过图中的边的邻近样本传播，直到图模型达到全局稳定状态为止。
#### 2）流型假设对输出值没有限制，因此比聚类假设适用范围更广，可用于更多类型的学习任务。
### 从本质上说，这三类假设是一致的,“相似的样本拥有相似的输出”，只是相互关注的重点不同。其中流行假设更具有普遍性。

## 三、半监督学习细化（利用数据分布信息进行半监督学习）

主动学习：若我们先使用有标记的样本数据集训练出一个学习器，再基于该学习器对未标记的样本进行预测，从中挑选出不确定性高或分类置信度低的样本来咨询专家并进行打标，最后使用扩充后的训练集重新训练学习器，这样便能大幅度降低标记成本，这便是主动学习（active learning），其目标是使用尽量少的/有价值的咨询来获得更好的性能。
半监督学习可进一步划分为纯半监督学习和直推学习，前者假定训练数据中未标记样本并非待预测数据，后者假设学习过程中所考虑未标记样本恰是待预测数据 ，学习目的是在这些未标记样本上获得最有泛化性能。
### 纯半监督学习基于“开放世界”，希望学得模型能适用于训练过程中未观测到的数据；直推学习基于“封闭世界”假设，仅试图对学习过程中观察到的未标记数据进行预测。

### 显然，主动学习需要与外界进行交互/查询/打标，其本质上仍然属于一种监督学习。事实上，无标记样本虽未包含标记信息，但它们与有标记样本一样都是从总体中独立同分布采样得到，因此它们所包含的数据分布信息对学习器的训练大有裨益。如何让学习过程不依赖外界的咨询交互，自动利用未标记样本所包含的分布信息的方法便是半监督学习（semi-supervised learning），即训练集同时包含有标记样本数据和未标记样本数据。

In [2]:
%%html
<img src="./t1.png", width=740, heigth=580>

## 四、其他方法
### 1、生成式方法
生成式方法（generative methods）是基于生成式模型的方法，即先对联合分布P（x,c）建模，从而进一步求解 P（c | x），此类方法假定样本数据服从一个潜在的分布，因此需要充分可靠的先验知识。例如：前面已经接触到的贝叶斯分类器与高斯混合聚类，都属于生成式模型。现假定总体是一个高斯混合分布，即由多个高斯分布组合形成，从而一个子高斯分布就代表一个类簇（类别）。

可以看出：基于生成式模型的方法十分依赖于对潜在数据分布的假设，即假设的分布要能和真实分布相吻合，否则利用未标记的样本数据反倒会在错误的道路上渐行渐远，从而降低学习器的泛化性能。因此，此类方法要求极强的领域知识和掐指观天的本领。

### 2、半监督SVM
监督学习中的SVM试图找到一个划分超平面，使得两侧支持向量之间的间隔最大，即“最大划分间隔”思想。对于半监督学习，半监督SVM（S3VM）则考虑超平面需穿过数据低密度的区域。
TSVM是半监督支持向量机中的最著名代表，其核心思想是：尝试为未标记样本找到合适的标记指派，使得超平面划分后的间隔最大化。
TSVM采用局部搜索的策略来进行迭代求解，即首先使用有标记样本集训练出一个初始SVM，接着使用该学习器对未标记样本进行打标，这样所有样本都有了标记，并基于这些有标记的样本重新训练SVM，之后再寻找易出错样本不断调整。整个算法流程如下所示：

### 3、过采样方法
​ 过采样：对小类的数据样本进行采样来增加小类的数据样本个数。  
​ 代表算法：SMOTE和ADASYN。  
​ SMOTE：通过对训练集中的小类数据进行插值来产生额外的小类样本数据。  
​ 新的少数类样本产生的策略：对每个少数类样本a，在a的最近邻中随机选一个样本b，然后在a、b之间的连线上随机选一点作为新合成的少数类样本。​ ADASYN：根据学习难度的不同，对不同的少数类别的样本使用加权分布，对于难以学习的少数类的样本，产生更多的综合数据。 通过减少类不平衡引入的偏差和将分类决策边界自适应地转移到困难的样本两种手段，改善了数据分布。  

## 先到这里，泛泛地讲没什么意思，还需要深刻再理解

## 参考文献

https://www.cnblogs.com/CJT-blog/p/10276310.html  